In [1]:
pip install psycopg2


The following command must be run outside of the IPython shell:

    $ pip install psycopg2

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [2]:
import psycopg2 as pg2

In [3]:
import numpy as np

In [26]:
conn = pg2.connect(host='localhost', database='dvdrental',user='postgres',password='indrika1')

In [27]:
cur = conn.cursor()

In [28]:
cur.execute('SELECT * FROM payment')

In [29]:
data=cur.fetchmany(10)

In [30]:
data2=np.array(data)
data2.shape

(10, 6)

In [31]:
data2[0,4]

Decimal('7.99')

In [32]:
#Create a new database with python

#First close the previous connection
conn.close()

#Create a new database

conn_new = pg2.connect(dbname='postgres', user='postgres')

conn_new.set_session(autocommit = True)

cur = conn_new.cursor()
cur.execute('CREATE DATABASE Temp_data1') # This creates a new 
                                         #database called Temp_data

#Close connection

conn_new.close()


ProgrammingError: database "temp_data1" already exists


In [33]:
#Open the new database, temp_data.  

conn = pg2.connect(database='temp_data1',user='postgres')


In [34]:
# Add a cursor and create a new table

cur = conn.cursor()

query1='''Create Table Temperature_target2 (
            entry integer
            , Instrument_name varchar(20)
            , Day int
            , Time_minutes int
            , Temp float
          )'''
cur.execute(query1)

In [37]:
#Add content to the table by reading a textfile

#First be sure that the csv file has the right permissions.
#On linux, use chmod 777
#On Windows cmd, use cacls myfile.txt /g everyone:f

query2 = '''
        COPY Temperature_target2 
        FROM '/users/priyashree/Documents/PostGreSQL/temp_data/Day2.csv' 
        DELIMITER ','
        CSV;
        '''
cur.execute(query2)

In [38]:
#read the data from the newly created table

query3 = '''
          SELECT * from Temperature_target2
          '''

cur.execute(query3)
cur.fetchall()

[(0, ' Condenser', 2, 0, 20.3),
 (1, ' Target', 2, 2, 22.0),
 (2, ' Target', 2, 7, 22.1),
 (3, ' Condenser', 2, 3, 19.9),
 (4, ' Target', 2, 0, 21.8),
 (5, ' Condenser', 2, 7, 20.5),
 (6, ' Condenser', 2, 6, 20.3)]

In [39]:
#Read all csv files from a directory

#First, create a new table

query4 = '''Create Table Temp_target_allDays_v2 (
            entry integer
            , Instrument_name varchar(20)
            , Day int
            , Time_minutes int
            , Temp float
          )'''
cur.execute(query4)


In [42]:
#Next, read all csv files

import os

folder_path = '/users/priyashree/Documents/PostGreSQL/temp_data/'

query5 = '''
          COPY Temp_target_allDays_v2
          FROM %(file_path)s
          DELIMITER ','
          CSV;
          '''

#Make a list of filenames in the folder_path
fnames = os.listdir(folder_path)

#Read all files with csv extension inside folder_path

print(fnames)


for file in fnames:
    if file.endswith(".csv"):
        #print(folder_path+file)
        cur.execute(query5, {'file_path':folder_path+file})

#Fetch all data from the newly created table
cur.execute('select * from Temp_target_allDays_v2')
cur.fetchall()  

['Day1.csv', 'Day2.csv']


[(0, 'Target', 3, 1, 22.2),
 (0, 'Target', 3, 1, 22.2),
 (0, ' Condenser', 1, 0, 20.1),
 (1, ' Target', 1, 3, 22.1),
 (2, ' Target', 1, 5, 22.0),
 (3, ' Condenser', 1, 3, 20.3),
 (4, ' Target', 1, 6, 21.9),
 (5, ' Condenser', 1, 5, 19.8),
 (6, ' Condenser', 1, 6, 20.2),
 (0, ' Condenser', 2, 0, 20.3),
 (1, ' Target', 2, 2, 22.0),
 (2, ' Target', 2, 7, 22.1),
 (3, ' Condenser', 2, 3, 19.9),
 (4, ' Target', 2, 0, 21.8),
 (5, ' Condenser', 2, 7, 20.5),
 (6, ' Condenser', 2, 6, 20.3)]

In [43]:
# add a new entry to Table Temp_target_allDays_v2

query6 = '''
          insert into Temp_target_allDays_v2
          
          (entry, Instrument_name, Day, Time_minutes, Temp)
          
          values (0, 'Target', 3, 1, 22.2)
          
          '''

cur.execute(query6)
cur.execute('select * from Temp_target_allDays_v2')
cur.fetchall() 

[(0, 'Target', 3, 1, 22.2),
 (0, 'Target', 3, 1, 22.2),
 (0, ' Condenser', 1, 0, 20.1),
 (1, ' Target', 1, 3, 22.1),
 (2, ' Target', 1, 5, 22.0),
 (3, ' Condenser', 1, 3, 20.3),
 (4, ' Target', 1, 6, 21.9),
 (5, ' Condenser', 1, 5, 19.8),
 (6, ' Condenser', 1, 6, 20.2),
 (0, ' Condenser', 2, 0, 20.3),
 (1, ' Target', 2, 2, 22.0),
 (2, ' Target', 2, 7, 22.1),
 (3, ' Condenser', 2, 3, 19.9),
 (4, ' Target', 2, 0, 21.8),
 (5, ' Condenser', 2, 7, 20.5),
 (6, ' Condenser', 2, 6, 20.3),
 (0, 'Target', 3, 1, 22.2)]

In [44]:
# Edit existing entry of the Table Temp_target_allDays_v2
#Edit entry # 4 from day 1

query7 = '''
         update Temp_target_allDays_v2
         set Temp = 21.89
         where (Day=1 and entry=4);
         '''
cur.execute(query7)
cur.execute('select * from Temp_target_allDays_v2 where Day=1')
cur.fetchall()


[(0, ' Condenser', 1, 0, 20.1),
 (1, ' Target', 1, 3, 22.1),
 (2, ' Target', 1, 5, 22.0),
 (3, ' Condenser', 1, 3, 20.3),
 (5, ' Condenser', 1, 5, 19.8),
 (6, ' Condenser', 1, 6, 20.2),
 (4, ' Target', 1, 6, 21.89)]

In [45]:
#Close the connection to the database at the end
#Otherwise, certain operations will be locked until the 
#connection is severed. 

cur.close()
conn.close()